### Write a SQL Query to fetch all the duplicate records in a table.

https://techtfq.com/blog/learn-how-to-write-sql-queries-practice-complex-sql-queries

In [ ]:
with add_row_number as (
	select *,
		row_number() over (
			partition by user_name
			order by user_id
		) as rn
	from users
	order by user_id
)
select user_id, user_name, email 
from add_row_number arn
where arn.rn != 1 ## select only first occuring user information


### Write a SQL query to fetch the second last record from employee table.

In [ ]:
with add_numbers as (
	select *, 
	  row_number() over (
	  	order by emp_id desc
	  ) as rn
from employee 
)
select emp_id, emp_name, dept_name 
from add_numbers
where rn = 2

### Write a SQL query to display only the details of employees who either earn the highest salary or the lowest salary in each department from the employee table.

In [ ]:

# first add the ranks to the employee table per department
with dense_rank_salary as (
select *,
	dense_rank() over (
		partition by dept_name 
		order by salary 
	) as dr
from employee e 
),
## get the min and max salary per department
min_max_salary as (
	select dept_name, 
			min(salary) as min_salary, 
			max(salary) as max_salary
	from dense_rank_salary e2 
	group by dept_name 

)

## join both table based on the department name and filter out employees whose salary matches and min, max 

select drs.emp_id, drs.emp_name, drs.dept_name, drs.salary, mms.min_salary, mms.max_salary
from dense_rank_salary drs
inner join min_max_salary mms
on drs.dept_name = mms.dept_name
where drs.salary = min_salary
or drs.salary = max_salary
order by drs.dept_name
 

In [ ]:
## get min and max salary per each department

with min_max_salary as (
select *,
	min(salary) over (
		partition by dept_name 
	) as min_salary, 
	max(salary) over (
		partition by dept_name 
	) as max_salary
from employee e 
)

# combine with employee table to filter out the other employees whose salary not in min, max range

select mms.emp_id, mms.emp_name, mms.dept_name, mms.salary, mms.min_salary, mms.max_salary
from min_max_salary mms
inner join employee e2
on mms.emp_id = e2.emp_id 
where e2.salary = min_salary
or e2.salary = max_salary
order by e2.dept_name, e2.salary

### From the doctors table, fetch the details of doctors who work in the same hospital but in different specialty.

select d1.*
from doctors d1
inner join doctors d2
on d1.id != d2.id
and d1.hospital = d2.hospital
and d1.speciality != d2.speciality


### From the login_details table, fetch the users who logged in consecutively 3 or more times.

with consecutive_users as (select *,
	lead(user_name, 1) over(
		order by login_id
	) as next_user_name,
	lead(user_name, 2) over(
		order by login_id
	) as next_to_next_user_name
from login_details )

select distinct user_name
from consecutive_users cu
where user_name = next_user_name
and user_name = next_to_next_user_name

### From the students table, write a SQL query to interchange the adjacent student names.

In [ ]:
select *,
case 
	when id%2 != 0  then lead(student_name, 1, student_name) over (order by id)
	when id%2 = 0 then lag(student_name, 1, student_name) over (order by id) 
end as new_student_name
from students 


### From the weather table, fetch all the records when London had extremely cold temperature for 3 consecutive days or more.


In [ ]:
with consecutive_temp as ( 
    select *,
case
    ## check for 2 consecutive days
	when temperature < 0 
	and lead(temperature) over (order by day) < 0 
	and lead(temperature, 2) over (order by day) < 0  then 'yes'
	
    ## check for previous day and next day 
	when temperature < 0 
	and lag(temperature) over (order by day) < 0 
	and lead(temperature) over (order by day) < 0  then 'yes'
	
    ## check for 2 previous days 
	when temperature < 0 
	and lag(temperature) over (order by day) < 0 
	and lag(temperature, 2) over (order by day) < 0  then 'yes'
	
	else 'no'
	
end as condition_met
from weather )

select id, city, temperature, day 
from consecutive_temp
where condition_met = 'yes'



#### From the following 3 tables (event_category, physician_speciality, patient_treatment), write a SQL query to get the histogram of specialties of the unique physicians who have done the procedures but never did prescribe anything.

In [ ]:


with prescribed_physician as(
    ## get the physcian who has done the Prescription
	select physician_id
	from patient_treatment pt
	inner join event_category ec
	on pt.event_name = ec.event_name
	where ec.category = 'Prescription'

)

## count the speciality of physcians who has done only Procedure
select speciality, count(speciality) as speciality_count
from PATIENT_TREATMENT pt
inner join PHYSICIAN_SPECIALITY ps
on pt.physician_id = ps.physician_id
inner join EVENT_CATEGORY ec 
on pt.event_name = ec.event_name
where ec.category = 'Procedure'
and pt.physician_id not in (select physician_id from prescribed_physician)
group by speciality

### Find the top 2 accounts with the maximum number of unique patients on a monthly basis.

In [ ]:

with unique_patients as (
	# count distinct users per month, account_id
    select to_char(date, 'month') as month, account_id, count(distinct patient_id) as no_of_unique_patients
	from patient_logs 
	group by month, account_id
	order by month, no_of_unique_patients desc, account_id
),
rank_accounts as (
    # rank the accounts based on total patients, account_id and partition by month
	select *,
	rank() over(
				partition by month
				order by no_of_unique_patients desc, account_id
				) as rnk
	from unique_patients
)
# select only top 2 accounts
select month, account_id, no_of_unique_patients
from rank_accounts
where rnk in (1, 2)